In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
data=pd.read_csv('/kaggle/input/churn-modellingcsv/Churn_Modelling.csv')
data.head()

In [ ]:
data.info()

In [ ]:
corrmat=data.corr()

top_corr_features = corrmat.index
plt.figure(figsize=(20,20))

g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap='RdYlGn')

In [ ]:
X=data.iloc[:,3:13]
Y=data.iloc[:,13]

In [ ]:
geography=pd.get_dummies(X['Geography'],drop_first=True)

In [ ]:
geography.head()

In [ ]:
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [ ]:
gender.head()

In [ ]:
X=X.drop(['Geography','Gender'],axis=1)

In [ ]:
X.head()

In [ ]:
X=pd.concat([X,geography,gender],axis=1)

In [ ]:
X.head()

In [ ]:
#Hyperparaemeter Optimization

params= {
    "learning_rate"   : [0.05, 0.10, 0.15, 0.20, 0.25, 0,30],
    "max_depth"       : [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma"           : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7]
}

In [ ]:
# Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() -start_time).total_seconds(), 3500)
        tmin, tsec = divmod(temp_sec, 60)
        print("\n Time taken %i hours %i minutes and %s seconds" % (thour, tmin, round(tsec,2)))

In [ ]:
classifier=xgboost.XGBClassifier()

In [ ]:
random_search = RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
from datetime import datetime

start_time= timer(None)
random_search.fit(X,Y)
timer(start_time)

In [ ]:
X.head()

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,X,Y,cv=10)

In [ ]:
score

In [ ]:
score.mean()